In [1]:
#@title Librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
from scipy.integrate import simpson, romberg, trapezoid
from scipy.special import roots_legendre


# Actividad 07: Integración

---
### Profesor: Juan Marcos Marín
### Nombre: Edgar Junior Córdoba Gamboa
*Métodos computacionales 2024-II*

---

#1
* Implemente una función para el **método de integración de Romberg** definiendo un límite de tolerancia de 1e-8 y/o un máximo de iteraciones de 10.

* Encuentre la integral para

$$\int_0^{\pi/4} dx\, e^{3x}\cdot \sin(x)$$

* Imprima su resultado y compare los valores dados por `scipy.integrate.romberg`

* Finalmente, encuentre el valor del error, hallando el valor exacto usando `sympy`



In [ ]:
import scipy.integrate as spi
f = lambda x: (np.exp(x)*(3))* np.sin(x)
a,b = 0, np.pi/4
def trape(f,a,b,n):
  h = (b-a)/n
  x = np.linspace(a,b, n+1)
  y = f(x)
  return (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

def room(f,a,b,tol=1e-8,max_iter=10):
  R = np.zeros((max_iter, max_iter))
  R[0, 0] = trape(f, a,b, 1)

  for i in range(1,max_iter):
    n = 2**i
    R[i,0]=trape(f,a,b,n)
    for k in range(1, i + 1):
     R[i, k] = (4**k * R[i, k - 1] - R[i - 1, k - 1]) / (4**k - 1)
    if abs(R[i, i] - R[i - 1, i - 1]) < tol:
      return R[i, i]

  return R[max_iter - 1, max_iter - 1]

valror = spi.romberg(f, a, b)
resultado = room(f,a,b)
print(f"Resultado con Romberg: {resultado:.6f}")
print(f'Resultado con scipy: {valror:.6f}')


Resultado con Romberg: 1.500000
Resultado con scipy: 1.500000


<ipython-input-20-d8b641a14b49>:24: DeprecationWarning: `scipy.integrate.romberg` is deprecated as of SciPy 1.12.0and will be removed in SciPy 1.15.0. Please use`scipy.integrate.quad` instead.
  valror = spi.romberg(f, a, b)


#2

* Usando los *métodos trapezoidal compuesto*, *simpson 1/3* y de *medio punto* encuentre la siguiente integral,

$$\int_e^{1+e} dx\, \frac{1}{x\ln x}$$

* Luego, haga un estudio de la convergencia en términos del valor de $h$ o de los sub-intervalos de la función. ¿Cuál es mejor?


In [ ]:
f = lambda x: 1/(np.log(x))*x
a,b = np.exp(1) , 1 + np.exp(1)
n = 100
def trapecio_compuesto(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

# Método de Simpson 1/3 compuesto
def simpson_tercio(f, a, b, n):
    if n % 2 == 1:  # Simpson requiere que n sea par
        n += 1
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (h / 3) * (y[0] + 4 * np.sum(y[1:n:2]) + 2 * np.sum(y[2:n-1:2]) + y[-1])

# Método del punto medio compuesto
def punto_medio(f, a, b, n):
    h = (b - a) / n
    x_medio = np.linspace(a + h / 2, b - h / 2, n)
    y = f(x_medio)
    return h * np.sum(y)

tra_r = trapecio_compuesto(f,a,b,n)
simtr = simpson_tercio(f,a,b,n)
pf = punto_medio(f,a,b,n)
print(f'Valor de Trapecio Compuesto: {tra_r:.2f}')
print(f'Valor de Simpson: {simtr:.2f}')
print(f'Valor Punto Medio: {pf:.2f}')


Valor de Trapecio Compuesto: 2.76
Valor de Simpson: 2.76
Valor Punto Medio: 2.76


In [ ]:
f = lambda x: 1 / (x * np.log(x))
a, b = np.exp(1), 1 + np.exp(1)
n = 100

def trapecio_compuesto(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

def simpson_tercio(f, a, b, n):
    if n % 2 == 1:
        n += 1
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (h / 3) * (y[0] + 4 * np.sum(y[1:n:2]) + 2 * np.sum(y[2:n-1:2]) + y[-1])

def punto_medio(f, a, b, n):
    h = (b - a) / n
    x_medio = np.linspace(a + h/2, b - h/2, n)
    y = f(x_medio)
    return h * np.sum(y)

tra_r = trapecio_compuesto(f, a, b, n)
simtr = simpson_tercio(f, a, b, n)
pf = punto_medio(f, a, b, n)

print(f'Valor de Trapecio Compuesto: {tra_r:.2f}')
print(f'Valor de Simpson: {simtr:.2f}')
print(f'Valor Punto Medio: {pf:.2f}')

Valor de Trapecio Compuesto: 0.27
Valor de Simpson: 0.27
Valor Punto Medio: 0.27


#3
Usando la siguiente función:



```python
def gauss_quad_standard(func, n):
    """
    Calcula la integral de una función en el intervalo [-1, 1]
    utilizando cuadratura gaussiana.

    Parameters:
    - func: La función a integrar.
    - n: Número de puntos para la cuadratura (grado del polinomio de Legendre).

    Returns:
    - Aproximación de la integral.
    """
    # Obtener raíces y pesos del polinomio de Legendre
    x, w = roots_legendre(n)

    # Evaluar la suma ponderada
    integral = np.sum(w * func(x))
    return integral
```

Modifique la función `gauss_quad_standard` de forma tal que no este restringida para $[-1,1]$ sino para cualquier intervalo $[a,b]$. Luego, encuentre la integral del *punto 2*.





In [ ]:
f = lambda x: 1 / (x * np.log(x))
a, b = np.exp(1), 1 + np.exp(1)  # Interval [e, e+1]

def gauss_quad_standard(func, a, b, n):

    x, w = roots_legendre(n)


    x_transformed = 0.5 * (b - a) * x + 0.5 * (a + b)


    integral = np.sum(w * func(x_transformed)) * 0.5 * (b - a)

    return integral

resultado = gauss_quad_standard(f, a, b, 100)
print("Resultado:", round(resultado, 2))


Resultado: 0.27


#4

Encuentra todas las raices para los polinomios de grado 3 y 4 de **Legendre** usando el Método de la Secante y Newton-Raphson.



```python
import sympy as sp
x = sp.Symbol('x')

# Polinomio de Legendre de grado n
Pn = sp.legendre(n, x)

```

y calcule los pesos $w_i$ de la cuadratura mediante la fórmula:
   $$
   w_i = \frac{2}{(1 - x_i^2) \left[P_n'(x_i)\right]^2},
   $$
   donde $P_n'(x)$ es la derivada del polinomio de Legendre $P_n(x)$.
